In [1]:
import os
import time
import warnings
import numpy as np
import random as rnd
import pandas as pd
from collections import defaultdict

# Librería Genética
from deap import base, creator, tools, algorithms

# Subfunciones de estimadores
from sklearn.base import clone
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/base.py][30]
from sklearn.base import is_classifier
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/base.py][535]
from sklearn.model_selection._validation import _fit_and_score
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_validation.py][346]
from sklearn.model_selection._search import BaseSearchCV
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py][386]
from sklearn.model_selection._search import check_cv
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_split.py][1866]
from sklearn.model_selection._search import _check_param_grid
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/model_selection/_search.py][343]
from sklearn.metrics.scorer import check_scoring
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/metrics/scorer.py][250]
from sklearn.utils.validation import _num_samples
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/validation.py][105]
from sklearn.utils.validation import indexable
# [https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/utils/validation.py][208]
from multiprocessing import Pool

# Selección para estimadores
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Metricas para estimadores
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

# Estimadores
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

## Funciones Genéticas

In [2]:
"""
Extracción
"""
from sklearn.utils import shuffle
import pandas as pd
import numpy as np


def _createDataset(frecuencias, individual):
    names_ = frecuencias[0].columns.values
    dataset = pd.DataFrame()
    dataset[names_[0]] = frecuencias[values[0]][names_[0]]
    dataset[names_[1]] = frecuencias[values[1]][names_[1]]
    dataset[names_[2]] = frecuencias[values[2]][names_[2]]
    dataset[names_[3]] = frecuencias[values[3]][names_[3]]
    dataset[names_[4]] = frecuencias[values[4]][names_[4]]
    dataset[names_[5]] = frecuencias[0][names_[5]]
    # separación de data en X,y 
    y = dataset[names_[5]]
    del dataset[names_[5]]
    X = dataset
    return X,y

seed = 7
frecuencias = []
frecuencias.append(pd.read_csv('Tx_0x01'))
frecuencias.append(pd.read_csv('Tx_0x02'))
frecuencias.append(pd.read_csv('Tx_0x03'))
frecuencias.append(pd.read_csv('Tx_0x04'))
frecuencias.append(pd.read_csv('Tx_0x05'))
frecuencias.append(pd.read_csv('Tx_0x06'))
frecuencias.append(pd.read_csv('Tx_0x07'))
frecuencias[0] = shuffle(frecuencias[0], random_state=seed).reset_index(drop=True)
frecuencias[1] = shuffle(frecuencias[1], random_state=seed).reset_index(drop=True)
frecuencias[2] = shuffle(frecuencias[2], random_state=seed).reset_index(drop=True)
frecuencias[3] = shuffle(frecuencias[3], random_state=seed).reset_index(drop=True)
frecuencias[4] = shuffle(frecuencias[4], random_state=seed).reset_index(drop=True)
frecuencias[5] = shuffle(frecuencias[5], random_state=seed).reset_index(drop=True)
frecuencias[6] = shuffle(frecuencias[6], random_state=seed).reset_index(drop=True)


In [3]:
values = np.random.randint(6,size=5) # orden randomización
X,y = _createDataset(frecuencias, values)

In [4]:
print("X")
display(X.head())
print("y")
display(y.head())

X


,Be01,Be02,Be03,Be04,Be05
0,-77,-61,-68,-63,-83
1,-79,-61,-69,-62,-89
2,-67,-69,-67,-71,-80
3,-70,-62,-66,-71,-80
4,-73,-59,-64,-70,-72


y


0     5
1     5
2    13
3     2
4     6
Name: Sector, dtype: int64

In [5]:
def _initIndividual(individuo, maxints):
	"""[Iniciar Individuo]
	Arguments:
		pcls {[creator.Individual]} -- [Iniciar individuo con indices aleatorios]
		maxints {[params_size]} -- [lista de máximos índices]
	Returns:
		[creator.Individual] -- [Creación de individuo]
	"""
	return individuo(rnd.randint(0, maxint) for maxint in maxints)

def _mutIndividual(individual, maxints, prob_mutacion):
	"""[Mutación Individuo]
	Arguments:
		individual {[creator.Individual]} -- [Individuo de población]
		maxints {[lista]} -- [lista de máximos índices]
		prob_mutacion {[float]} -- [probabilidad de mutación del gen]
	Returns:
		[creator.Individual] -- [Individuo mutado]
	"""
	for i in range(len(maxints)):
		if rnd.random() < prob_mutacion:
			individual[i] = rnd.randint(0, maxints[i])
	return individual,

def _cxIndividual(ind1, ind2, prob_cruce):
	"""[Cruce de Individuos]
	Arguments:
		ind1 {[creator.Individual]} -- [Individuo 1]
		ind2 {[creator.Individual]} -- [Individuo 2]
		indpb {[float]} -- [probabilidad de emparejar]
		gene_type {[list]} -- [tipos de dato de los parámetros, CATEGORICO o NUMERICO]
	Returns:
		[creator.Individual,creator.Individual] -- [nuevos Individuos]
	"""
	CATEGORICO = 1  # int o str
	NUMERICO = 2  # float
	for i in range(len(ind1)):
		if rnd.random() < prob_cruce:
			sorted_ind = sorted([ind1[i], ind2[i]])
			ind1[i] = rnd.randint(sorted_ind[0], sorted_ind[1])
			ind2[i] = rnd.randint(sorted_ind[0], sorted_ind[1])
	return ind1, ind2

def _individual_to_params(individual, frecuencias):
	names_ = frecuencias[0].columns.values
	dataset = pd.DataFrame()
	dataset[names_[0]] = frecuencias[individual[0]][names_[0]]
	dataset[names_[1]] = frecuencias[individual[1]][names_[1]]
	dataset[names_[2]] = frecuencias[individual[2]][names_[2]]
	dataset[names_[3]] = frecuencias[individual[3]][names_[3]]
	dataset[names_[4]] = frecuencias[individual[4]][names_[4]]
	dataset[names_[5]] = frecuencias[0][names_[5]]
	# separación de data en X,y 
	y = dataset[names_[5]]
	del dataset[names_[5]]
	X = dataset
	return X,y


def _evalFunction(individual, frecuencias, scorer, num_folds, uniform, fit_params,
				verbose=0, error_score='raise', score_cache={}):
	"""[Evaluación del modelo]
	Arguments:
		individual {[creator.Individual]} -- [Individuo]
		frecuencias {[list]} -- [lista de dataframes]
		X {[array]} -- [Input]
		y {[array]} -- [Output]
		scorer {[string]} -- [Parámetro de evaluación, precisión]
		cv {[int | cross-validation]} -- [Especificación de los folds]
		uniform {[boolean]} -- [True hace que la data se distribuya uniformemente en los folds]
		fit_params {[dict | None]} -- [parámetros para estimator.fit]
	Keyword Arguments:
		verbose {integer} -- [Mensajes de descripción] (default: {0})
		error_score {numerico} -- [valor asignado si ocurre un error en fitting] (default: {'raise'})
		score_cache {dict} -- [description] (default: {{}})
	"""
	X, y = _individual_to_params(individual, frecuencias)
	score = 0
	n_test = 0
	paramkey = str(individual)
	if paramkey in score_cache:
		score = score_cache[paramkey]
	else:
		kfold = KFold(n_splits=num_folds)
		#cv_results = cross_val_score(estimator, X, y, cv=kfold, scoring=scoring)
		cv_results = cross_val_score(individual.est, X, y, cv=kfold, scoring=scorer)
		score = np.mean(cv_results)
		score_cache[paramkey] = score
	return (score,)

In [6]:
class EvolutiveSearchCV:
	def __init__(self, estimator, scoring=None, num_folds=4,
				refit=True, verbose=False, population_size=50,
				gene_mutation_prob=0.2, gene_crossover_prob=0.5,
				tournament_size=3, generations_number=10, gene_type=None,
				n_jobs=1, uniform=True, error_score='raise',
				fit_params={}):
		# Parámetros iniciales
		self.estimator = estimator
		#self.params = params
		self.scoring = scoring
		self.num_folds = num_folds
		self.refit = refit
		self.verbose = verbose
		self.population_size = population_size
		self.gene_mutation_prob = gene_mutation_prob
		self.gene_crossover_prob = gene_crossover_prob
		self.tournament_size = tournament_size
		self.generations_number = generations_number
		self.gene_type = gene_type
		self.n_jobs = n_jobs
		self.uniform = uniform
		self.error_score = error_score
		self.fit_params = fit_params
		# Parámetros adicionales
		self._individual_evals = {}
		self.all_history_ = None
		self.all_logbooks_ = None
		self._cv_results = None
		self.best_score_ = None
		self.best_params_ = None
		self.scorer_ = None
		self.score_cache = {}
		# Fitness [base.Fitness], objetivo 1
		creator.create("FitnessMax", base.Fitness, weights=(1.0,))
		# Individuo [list], parámetros:est, FinessMax
		creator.create("Individual", list, est=clone(self.estimator), fitness=creator.FitnessMax)
	#@property
	def cv_results_(self):
		if self._cv_results is None:
			out = defaultdict(list)
			gen = self.all_history_
			# Get individuals and indexes, their list of scores,
			# and additionally the name_values for this set of parameters
			idxs, individuals, each_scores = zip(*[(idx, indiv, np.mean(indiv.fitness.values))
											for idx, indiv in list(gen.genealogy_history.items())
											if indiv.fitness.valid and not np.all(np.isnan(indiv.fitness.values))])
			#name_values, _, _ = _get_param_types_maxint(self.params)
			# Add to output
			#out['param_index'] += [p] * len(idxs)
			out['index'] += idxs
			#out['params'] += [_individual_to_params(indiv, name_values) for indiv in individuals]
			out['params'] += [str(np.add(indiv,1)) for indiv in individuals]
			out['mean_test_score'] += [np.nanmean(scores)*100 for scores in each_scores]
			out['std_test_score'] += [np.nanstd(scores)*100 for scores in each_scores]
			out['min_test_score'] += [np.nanmin(scores) for scores in each_scores]
			out['max_test_score'] += [np.nanmax(scores) for scores in each_scores]
			out['nan_test_score?'] += [np.any(np.isnan(scores)) for scores in each_scores]
			self._cv_results = out
		return self._cv_results
	@property
	def best_index_(self):
		return np.argmax(self.cv_results_['max_test_score'])
	# fit y refit general
	def fit(self, frecuencias):
		self.best_estimator_ = None
		self.best_mem_score_ = float("-inf")
		self.best_mem_params_ = None
		#_check_param_grid(self.params)
		self._fit(frecuencias)
		if self.refit:
			self.best_estimator_ = clone(self.estimator)
			#self.best_estimator_.set_params(**self.best_mem_params_)
			if self.fit_params is not None:
				self.best_estimator_.fit(X, y, **self.fit_params)
			else:
				self.best_estimator_.fit(X, y)
	# fit individual
	def _fit(self, frecuencias):
		self._cv_results = None  # Indicador de necesidad de actualización
		self.scorer_ = check_scoring(self.estimator, scoring=self.scoring)
		#n_samples = _num_samples(X)
		# verificar longitudes x,y 
		#if _num_samples(y) != n_samples:
		#	raise ValueError('Target [y], data [X] dont agree')
		#cv = check_cv(self.cv, y=y, classifier=is_classifier(self.estimator))
		toolbox = base.Toolbox()
		# name_values = lista de parametros, gene_type = [1:categorico; 2:numérico], maxints = size(parametros)
		#name_values, self.gene_type, maxints = _get_param_types_maxint(parameter_dict)
		maxints = [6]*5
		#if self.verbose:
		#	print("Tipos: %s, rangos: %s" % (self.gene_type, maxints))
		# registro de función Individuo
		toolbox.register("individual", _initIndividual, creator.Individual, maxints=maxints)
		# registro de función Población
		toolbox.register("population", tools.initRepeat, list, toolbox.individual)
		# Paralelísmo, create pool
		if not isinstance(self.n_jobs, int):
			self.n_jobs=1
		pool = Pool(self.n_jobs)
		toolbox.register("map", pool.map)
		# registro de función Evaluación
		toolbox.register("evaluate", _evalFunction,
						frecuencias=frecuencias,
						scorer=self.scorer_, num_folds=4, uniform=self.uniform,
						error_score=self.error_score, fit_params=self.fit_params,
						score_cache=self.score_cache)
		# registro de función Cruce
		toolbox.register("mate", _cxIndividual, prob_cruce=self.gene_crossover_prob)
		# registro de función Mutación
		toolbox.register("mutate", _mutIndividual, prob_mutacion=self.gene_mutation_prob, maxints=maxints)
		# registro de función Selección
		toolbox.register("select", tools.selTournament, tournsize=self.tournament_size)
		# Creación de Población
		pop = toolbox.population(n=self.population_size)
		# Mejor Individuo que ha existido
		hof = tools.HallOfFame(1)
		# Stats
		stats = tools.Statistics(lambda ind: ind.fitness.values)
		stats.register("avg", np.nanmean)
		stats.register("min", np.nanmin)
		stats.register("max", np.nanmax)
		stats.register("std", np.nanstd)
		# Genealogía
		hist = tools.History()
		# Decoración de operadores de variaznza
		toolbox.decorate("mate", hist.decorator)
		toolbox.decorate("mutate", hist.decorator)
		hist.update(pop)
		# Posibles combinaciones
		if self.verbose:
			print('--- Evolve in {0} possible combinations ---'.format(np.prod(np.array(maxints) + 1)))
		pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=self.gene_crossover_prob, 
										mutpb=self.gene_mutation_prob,
										ngen=self.generations_number, 
										stats=stats,
										halloffame=hof, 
										verbose=self.verbose)
		#pop, logbook = algorithms.eaGenerateUpdate(toolbox,
		#								ngen=self.generations_number, stats=stats,
		#								halloffame=hof, verbose=self.verbose)
		# Save History
		self.all_history_ = hist
		self.all_logbooks_ = logbook
		# Mejor score y parametros
		current_best_score_ = hof[0].fitness.values[0]
		current_best_params_ = str(hof[0]) #_individual_to_params(hof[0], name_values)
		#if self.verbose:
		#	print("Best individual is: %s\nwith fitness: %s" % (
		#		current_best_params_, current_best_score_))
		if current_best_score_ > self.best_mem_score_:
			self.best_mem_score_ = current_best_score_
			self.best_mem_params_ = current_best_params_
		# fin paralelización, close pool
		pool.close()
		pool.join()
		self.best_score_ = current_best_score_
		self.best_params_ = current_best_params_


In [7]:
"""
maxints = [5]*5
scoring = "accuracy"
num_folds = 4
maxints = [5]*5
model = KNeighborsClassifier()
def __init__(self, estimador, scoring=None, num_folds=4,
				refit=True, verbose=False, population_size=50,
				gene_mutation_prob=0.2, gene_crossover_prob=0.5,
				tournament_size=3, generations_number=10, gene_type=None,
				n_jobs=1, uniform=True, error_score='raise',
				fit_params={}):
"""
gs = EvolutiveSearchCV(estimator=KNeighborsClassifier(), scoring="accuracy", num_folds=4, n_jobs=4,
                        verbose=True, refit=True, 
                        population_size=60, 
                        gene_mutation_prob=0.2, 
                        gene_crossover_prob=0.5,
                        tournament_size=3,
                        generations_number=8)

In [8]:
gs.fit(frecuencias)
h = gs.cv_results_()

--- Evolve in 16807 possible combinations ---
gen	nevals	avg     	min   	max     	std      
0  	60    	0.782403	0.6575	0.873333	0.0496348
1  	38    	0.810306	0.674167	0.873333	0.0410566
2  	40    	0.837   	0.738333	0.895833	0.0346745
3  	32    	0.855597	0.765   	0.9175  	0.0334381
4  	39    	0.875361	0.7725  	0.925   	0.0298474
5  	32    	0.891653	0.828333	0.925   	0.0228575
6  	26    	0.904528	0.805833	0.935833	0.0230561
7  	36    	0.914681	0.795   	0.935833	0.0224831
8  	41    	0.925097	0.898333	0.935833	0.00626404


In [9]:
resultados = {}
resultados['Frecuencias'] = dict(h)['params']
resultados['Precision'] = dict(h)['mean_test_score']
df = pd.DataFrame(resultados)
df.drop_duplicates(subset=['Frecuencias', 'Precision']).sort_values(['Precision'],ascending=False).head(25)

,Frecuencias,Precision
271,[4 1 3 3 7],93.583333
238,[6 1 3 3 7],93.583333
269,[6 1 3 3 6],93.583333
265,[6 1 3 3 3],93.583333
197,[4 1 3 3 4],92.500000
224,[6 1 3 3 3],92.500000
154,[6 1 3 3 4],92.500000
270,[6 1 3 3 5],92.500000
242,[6 1 5 3 3],91.750000
200,[6 1 3 3 7],91.750000


In [18]:
gs2 = EvolutiveSearchCV(estimator=KNeighborsClassifier(algorithm='ball_tree'), scoring="accuracy", 
                        num_folds=4, n_jobs=4,
                        verbose=True, refit=True, 
                        population_size=60, 
                        gene_mutation_prob=0.2, 
                        gene_crossover_prob=0.5,
                        tournament_size=3,
                        generations_number=10)
gs2.fit(frecuencias)
h2 = gs2.cv_results_()
resultados = {}
resultados['Frecuencias'] = dict(h2)['params']
resultados['Precision'] = dict(h2)['mean_test_score']
df = pd.DataFrame(resultados)
df.drop_duplicates(subset=['Frecuencias', 'Precision']).sort_values(['Precision'],ascending=False).head(25)

/usr/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


--- Evolve in 16807 possible combinations ---
gen	nevals	avg     	min     	max     	std      
0  	60    	0.775486	0.639167	0.918333	0.0597674
1  	31    	0.809778	0.734167	0.918333	0.0459438
2  	43    	0.843306	0.753333	0.918333	0.038769 
3  	30    	0.866222	0.7175  	0.918333	0.0445641
4  	38    	0.892569	0.811667	0.934167	0.0300791
5  	37    	0.904   	0.7575  	0.934167	0.0358313
6  	38    	0.914194	0.810833	0.934167	0.0264028
7  	34    	0.923403	0.883333	0.934167	0.0114112
8  	32    	0.923097	0.789167	0.934167	0.0269069
9  	25    	0.929264	0.826667	0.934167	0.0196227
10 	39    	0.929944	0.856667	0.934167	0.0149582


,Frecuencias,Precision
342,[6 1 7 2 7],93.416667
278,[4 1 3 3 7],93.416667
211,[6 1 3 3 3],93.416667
212,[5 1 3 3 7],93.416667
245,[6 1 3 1 7],93.416667
153,[6 1 3 3 7],93.416667
247,[7 1 3 3 7],93.416667
249,[6 1 6 3 7],93.416667
277,[6 7 3 6 7],93.416667
279,[3 1 1 3 7],93.416667


In [19]:
gs3 = EvolutiveSearchCV(estimator=KNeighborsClassifier(algorithm='kd_tree'), scoring="accuracy", 
                        num_folds=4, n_jobs=4,
                        verbose=True, refit=True, 
                        population_size=60, 
                        gene_mutation_prob=0.2, 
                        gene_crossover_prob=0.5,
                        tournament_size=3,
                        generations_number=10)
gs3.fit(frecuencias)
h3 = gs3.cv_results_()
resultados = {}
resultados['Frecuencias'] = dict(h3)['params']
resultados['Precision'] = dict(h3)['mean_test_score']
df = pd.DataFrame(resultados)
df.drop_duplicates(subset=['Frecuencias', 'Precision']).sort_values(['Precision'],ascending=False).head(25)

/usr/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


--- Evolve in 16807 possible combinations ---
gen	nevals	avg     	min  	max  	std      
0  	60    	0.785833	0.655	0.895	0.0534062
1  	40    	0.810986	0.711667	0.895833	0.0467196
2  	38    	0.839097	0.765   	0.895833	0.0321834
3  	27    	0.860847	0.676667	0.9025  	0.0349137
4  	36    	0.865431	0.75    	0.906667	0.0308489
5  	36    	0.882194	0.794167	0.935833	0.0212911
6  	42    	0.884306	0.779167	0.935833	0.0262304
7  	31    	0.891014	0.779167	0.935833	0.0293971
8  	38    	0.904708	0.809167	0.935833	0.0328503
9  	43    	0.922681	0.839167	0.935833	0.021595 
10 	35    	0.929097	0.8225  	0.935833	0.0218175


,Frecuencias,Precision
362,[2 2 3 3 7],93.583333
359,[6 1 3 3 1],93.583333
328,[6 2 3 3 6],93.583333
183,[6 1 3 3 7],93.583333
205,[6 1 5 3 2],93.583333
227,[6 3 3 3 5],93.583333
271,[6 3 3 3 7],93.583333
254,[7 1 3 3 7],93.583333
255,[6 1 3 3 7],91.166667
181,[7 1 3 3 7],91.166667


In [21]:
gs4 = EvolutiveSearchCV(estimator=KNeighborsClassifier(algorithm='brute'), scoring="accuracy",
                        num_folds=4, n_jobs=4,
                        verbose=True, refit=True, 
                        population_size=60, 
                        gene_mutation_prob=0.2, 
                        gene_crossover_prob=0.5,
                        tournament_size=3,
                        generations_number=10)
gs4.fit(frecuencias)
h4 = gs4.cv_results_()
resultados = {}
resultados['Frecuencias'] = dict(h4)['params']
resultados['Precision'] = dict(h4)['mean_test_score']
df = pd.DataFrame(resultados)
df.drop_duplicates(subset=['Frecuencias', 'Precision']).sort_values(['Precision'],ascending=False).head(25)

/usr/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/usr/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


--- Evolve in 16807 possible combinations ---
gen	nevals	avg   	min  	max   	std      
0  	60    	0.7755	0.645	0.9275	0.0500456
1  	37    	0.815958	0.724167	0.9275	0.0419015
2  	36    	0.840431	0.735833	0.9275	0.0381607
3  	40    	0.867861	0.789167	0.9275	0.0287391
4  	28    	0.885986	0.795   	0.929167	0.0271375
5  	41    	0.894389	0.779167	0.929167	0.0291583
6  	42    	0.910903	0.7775  	0.929167	0.0249403
7  	34    	0.920278	0.849167	0.935   	0.0154805
8  	38    	0.9185  	0.805   	0.935   	0.0250628
9  	38    	0.922972	0.848333	0.935   	0.0182075
10 	36    	0.925653	0.7975  	0.935   	0.0219586


,Frecuencias,Precision
329,[3 1 3 3 7],93.500000
264,[6 1 3 3 7],93.500000
348,[6 1 3 3 5],93.500000
324,[6 1 3 3 6],93.500000
287,[6 1 2 3 1],92.916667
152,[6 1 3 3 4],92.916667
286,[5 3 3 3 1],92.916667
141,[6 1 3 3 2],92.916667
194,[6 1 3 3 1],92.916667
147,[6 1 3 3 3],92.916667
